In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Step 1: Load and preprocess the dataset
df = pd.read_csv(r'D:\Y\DATA\PREPROCESS\BookA.csv', parse_dates=['Date'], index_col=['Date'])

# Step 2: Split the data into train and test sets
features = df[['PM2.5', 'PM10', 'NO', 'NO2', 'SO2', 'CO', 'Ozone', 'RH', 'WS', 'WD', 'SR']].values
targets = df[['PM2.5']].values

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, shuffle=False)

# Step 3: Scale the data
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Step 4: Reshape the data for LSTM input
time_steps = 10
num_features = features.shape[1]

def reshape_data(X, y, time_steps):
    X_reshaped = []
    y_reshaped = []
    for i in range(time_steps, len(X)):
        X_reshaped.append(X[i - time_steps:i, :])
        y_reshaped.append(y[i])
    return np.array(X_reshaped), np.array(y_reshaped)

X_train_reshaped, y_train_reshaped = reshape_data(X_train, y_train, time_steps)
X_test_reshaped, y_test_reshaped = reshape_data(X_test, y_test, time_steps)

# Step 5: Define the fitness function for PSO optimization
def fitness_function(position):
    num_lstm_units = int(position[0])
    learning_rate = position[1]

    model = Sequential()
    model.add(LSTM(num_lstm_units, activation='relu', input_shape=(time_steps, num_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    history = model.fit(X_train_reshaped, y_train_reshaped, epochs=50, batch_size=32,
                        validation_split=0.2, callbacks=[early_stopping])

    loss = model.evaluate(X_test_reshaped, y_test_reshaped)
    return loss

# Step 6: Implement Particle Swarm Optimization
class Particle:
    def __init__(self, position, velocity):
        self.position = position
        self.velocity = velocity
        self.best_position = position
        self.best_fitness = float('inf')

class PSO:
    def __init__(self, num_particles, num_dimensions, bounds, max_iterations):
        self.num_particles = num_particles
        self.num_dimensions = num_dimensions
        self.bounds = bounds
        self.max_iterations = max_iterations
        self.global_best_position = None
        self.global_best_fitness = float('inf')
        self.particles = []

    def initialize_particles(self):
        for _ in range(self.num_particles):
            position = np.random.uniform(self.bounds[0], self.bounds[1], self.num_dimensions)
            velocity = np.zeros(self.num_dimensions)
            particle = Particle(position, velocity)
            self.particles.append(particle)

    def update_particle_velocity(self, particle):
        inertia_weight = 0.5
        cognitive_weight = 1.0
        social_weight = 1.0

        r1 = np.random.rand(self.num_dimensions)
        r2 = np.random.rand(self.num_dimensions)

        cognitive_component = cognitive_weight * r1 * (particle.best_position - particle.position)
        social_component = social_weight * r2 * (self.global_best_position - particle.position)
        particle.velocity = inertia_weight * particle.velocity + cognitive_component + social_component

    def update_particle_position(self, particle):
        particle.position += particle.velocity
        particle.position = np.clip(particle.position, self.bounds[0], self.bounds[1])

    def evaluate_fitness(self, particle):
        fitness = fitness_function(particle.position)
        return fitness

    def update_global_best(self):
        for particle in self.particles:
            if particle.best_fitness < self.global_best_fitness:
                self.global_best_fitness = particle.best_fitness
                self.global_best_position = particle.best_position

    def run(self):
        self.initialize_particles()

        for iteration in range(self.max_iterations):
            for particle in self.particles:
                fitness = self.evaluate_fitness(particle)

                if fitness < particle.best_fitness:
                    particle.best_fitness = fitness
                    particle.best_position = particle.position

                if fitness < self.global_best_fitness:
                    self.global_best_fitness = fitness
                    self.global_best_position = particle.position

                self.update_particle_velocity(particle)
                self.update_particle_position(particle)

            self.update_global_best()

            print("Iteration:", iteration + 1, "Best Fitness:", self.global_best_fitness)

        print("Optimization finished!")
        print("Best Fitness:", self.global_best_fitness)
        print("Best Position:", self.global_best_position)

# Step 7: Run the hybrid model using PSO and LSTM
num_particles = 20
num_dimensions = 2
bounds = [(16, 128), (0.001, 0.1)]
#bounds = [(16), (0.001)]
max_iterations = 50

pso = PSO(num_particles, num_dimensions, bounds, max_iterations)
pso.run()

# Step 8: Evaluate the final LSTM model with the optimized hyperparameters
best_position = pso.global_best_position

num_lstm_units = int(best_position[0])
learning_rate = best_position[1]

model = Sequential()
model.add(LSTM(num_lstm_units, activation='relu', input_shape=(time_steps, num_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train_reshaped, y_train_reshaped, epochs=50, batch_size=32,
                    validation_split=0.2, callbacks=[early_stopping])

loss = model.evaluate(X_test_reshaped, y_test_reshaped)
predictions = model.predict(X_test_reshaped)

# Step 9: Plot the training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()


Epoch 1/50
987/987 [==============================] - 3s 2ms/step - loss: 11017.0752 - val_loss: 5516.4146
Epoch 2/50
987/987 [==============================] - 2s 2ms/step - loss: 6010.0620 - val_loss: 4347.8159
Epoch 3/50
987/987 [==============================] - 2s 2ms/step - loss: 5053.2524 - val_loss: 4188.2920
Epoch 4/50
987/987 [==============================] - 2s 2ms/step - loss: 4599.1680 - val_loss: 4130.8022
Epoch 5/50
987/987 [==============================] - 2s 2ms/step - loss: 4275.4209 - val_loss: 3567.8828
Epoch 6/50
987/987 [==============================] - 2s 2ms/step - loss: 3883.7441 - val_loss: 3164.5945
Epoch 7/50
987/987 [==============================] - 2s 2ms/step - loss: 3685.1284 - val_loss: 2726.9487
Epoch 8/50
987/987 [==============================] - 2s 2ms/step - loss: 3277.1475 - val_loss: 2595.6384
Epoch 9/50
987/987 [==============================] - 2s 2ms/step - loss: 3021.2678 - val_loss: 2568.4783
Epoch 10/50
987/987 [========================